In [2]:
# import functions
import idaes
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom
# import HFC_IL mixture properties
from HFC32_emimTf2N_PR import configuration

# Create flash model

In [3]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

2021-03-24 10:53:27 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
The initial DOF is 7


In [4]:
# Read data
data = pd.read_csv('R32_emimTf2N.csv')
data_283= data[0:7]
data_298 = data[7:15]
data_323 = data[15:23]
data_348 = data[23:31]
print(len(data_298))
# print(data_298)

8


In [5]:
#generalize calc P function

In [6]:
# make function general
def calc_outlet_xP(data,HFC,IL, k21,k12,verbose=False):
    '''Inputs
            Data - csv file for x,T,P data HFC-IL
            HFC  - string with name of HFC (i.e.'R125')
            IL   - string with name of IL (i.e. 'emimTf2N')
            k21  - binary parameter IL-HFC
            k12  - binary parameter HFC-IL
            
            '''
    # create zeros arrays to be filled with x1 and P1
    HFC_x = np.zeros((len(data)))
    P = np.zeros((len(data)))
    
    # model constraints
    m.fs.liq = Param(mutable=True,default=0.040)
    m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emim_Tf2N"] == m.fs.liq)

    for i in range(len(data)):
        m.fs.liq = data['x_'+IL].iloc[i]
        m.fs.F101.inlet.flow_mol.fix(1) 
        m.fs.F101.inlet.temperature.fix(float(data["temperature"].iloc[i]))
        m.fs.F101.inlet.pressure.fix(float(data["pressure"].iloc[i]))
        m.fs.F101.inlet.mole_frac_comp[0, HFC].fix(float(data['x_'+HFC].iloc[i])+0.1)
        m.fs.F101.inlet.mole_frac_comp[0, IL].fix(float(1-(data['x_'+HFC].iloc[i]+0.1)))
        m.fs.F101.vap_outlet.temperature.fix(float(data["temperature"].iloc[i]))
        m.fs.properties.PR_kappa[IL, HFC].fix(k21) # (-0.20093)
        m.fs.properties.PR_kappa[HFC, IL].fix(k12) # (-0.05619)

        if verbose:
            DOF_final = degrees_of_freedom(m)
            print("The final DOF is {0}".format(DOF_final))

        # solver
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        solver = SolverFactory('ipopt')
        solver.options = {'tol': 1e-6}
        status = solver.solve(m, tee = False)
    #     m.fs.F101.report()
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            HFC_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,HFC]) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
        else:
            print('Infeasible.')
    return (HFC_x,P)

In [8]:
R125_x,P= calc_outlet_xP(data,'R_125','emimTf2N',-0.034795087689,-0.015923346845, verbose=True)

KeyError: "Index '(0, 'emim_Tf2N')' is not valid for indexed component 'fs.F101.split._Liq_mole_frac_comp_ref'"